# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
# Setting temperature to 0.75 to achieve a balanced between focus/deterministic (0.0) and creativity (1.0).
llm = ChatOpenAI(temperature=0.75)


In [17]:
prompt = ChatPromptTemplate.from_template(
    "Write a product description about a {product}. Account for it being weapons from warhammer 40.000 franchise."
    "Don't mention the franchise itself. Be short and concise."
)

In [18]:
chain = prompt | llm

In [19]:
product = "Boltgun"
chain.invoke(product)

AIMessage(content='The Boltgun is a powerful and versatile weapon used by soldiers in the grim darkness of the far future. With its rapid-firing bolts and devastating firepower, it is a staple weapon for warriors in battle against the enemies of mankind. Whether used by Space Marines or Imperial Guardsmen, the Boltgun is a reliable and deadly tool for bringing justice to the galaxy.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 41, 'total_tokens': 113, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fcd384f5-4510-4605-afb0-e70d69a2c04c-0', usage_metadata={'input_tokens': 41, 'output_tokens': 72, 'total_tokens': 113, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

## SimpleSequentialChain

In [20]:
from langchain.chains import SimpleSequentialChain

In [25]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Write a product description about a {product}. Account for it being a weapon from warhammer 40.000 franchise."
    "Don't mention the franchise itself. Be short and concise."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt, verbose=True)

In [29]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the description {descrtiption}, give an example about what the product would be effective against."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt, verbose=True)

In [30]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [31]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a product description about Boltgun. Account for it being weapons from warhammer 40.000 franchise.Don't mention the franchise itself. Be short and concise.

> Finished chain.
The Boltgun is a powerful and versatile firearm used by the elite soldiers of the Imperium. With its rapid firing speed and explosive ammunition, it is capable of taking down even the toughest enemies on the battlefield. Perfect for close-quarters combat and reliable in any situation, the Boltgun is a must-have weapon for any warrior fighting in the grim darkness of the far future.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Based on the description The Boltgun is a powerful and versatile firearm used by the elite soldiers of the Imperium. With its rapid firing speed and explosive ammunition, it is capable of taking down even the toughest enemies on the battlefield. Perfec

{'input': 'Boltgun',
 'output': 'The Boltgun would be highly effective against heavily armored enemies such as Chaos Space Marines or Ork Nobz. Its explosive ammunition can easily penetrate through thick armor, allowing the wielder to quickly eliminate these formidable foes in battle. The rapid firing speed of the Boltgun also ensures that multiple shots can be fired in quick succession, increasing the likelihood of hitting the target and taking them down swiftly. In the hands of a skilled warrior, the Boltgun is a deadly weapon that can turn the tide of a battle in favor of the Imperium.'}

**Repeat the above twice for different products**

In [32]:
# Product 2
product = "plasma rifle"

overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a product description about plasma rifle. Account for it being weapons from warhammer 40.000 franchise.Don't mention the franchise itself. Be short and concise.

> Finished chain.
Introducing the Plasma Rifle, a powerful and devastating weapon that harnesses the energy of superheated plasma to incinerate enemy targets with precision and efficiency. The perfect choice for those seeking a formidable firearm on the battlefield.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Based on the description Introducing the Plasma Rifle, a powerful and devastating weapon that harnesses the energy of superheated plasma to incinerate enemy targets with precision and efficiency. The perfect choice for those seeking a formidable firearm on the battlefield., give an example about what the product would be effective against.

> Finished chain.
The Plasma Rifle would

{'input': 'plasma rifle',
 'output': 'The Plasma Rifle would be highly effective against heavily armored enemies or vehicles, as the superheated plasma would be able to easily penetrate through their defenses and quickly disable or destroy them.'}

In [33]:
# Product 3
product = "fusion cannon"

overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Write a product description about fusion cannon. Account for it being weapons from warhammer 40.000 franchise.Don't mention the franchise itself. Be short and concise.

> Finished chain.
The fusion cannon is a devastating weapon capable of unleashing powerful blasts of energy to decimate enemy forces on the battlefield. With its advanced technology and high energy output, this cannon is a formidable choice for those seeking to turn the tide of war in their favor. Trust in the fusion cannon to deliver devastating blows to all who stand in your way.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Based on the description The fusion cannon is a devastating weapon capable of unleashing powerful blasts of energy to decimate enemy forces on the battlefield. With its advanced technology and high energy output, this cannon is a formidable choice for those seekin

{'input': 'fusion cannon',
 'output': 'The fusion cannon would be highly effective against armored enemy vehicles, such as tanks and armored personnel carriers. Its powerful blasts of energy would be able to penetrate through their thick armor, causing massive damage and quickly neutralizing the threat they pose on the battlefield. Trust in the fusion cannon to deliver devastating blows and ensure victory against even the toughest of enemy armored vehicles.'}

## SequentialChain

In [34]:
from langchain.chains import SequentialChain

In [35]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate this {review} into spanish."
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="review_es",
                     verbose=True
                    )


In [36]:
second_prompt = ChatPromptTemplate.from_template(
    "Based on {review_es}, summarize this review in less than 12 words in spanish."
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="review_es_short",
                     verbose=True
                    )


In [37]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate {review_es_short} into english"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="review_en_short",
                       verbose=True
                      )


In [38]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Rate the quality of the translation of {review_es_short} into {review_en_short} from zero to ten."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="translation_rating",
                      verbose=True
                     )


In [39]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["review_es_short", "review_en_short", "translation_rating"],
    verbose=True
)

In [41]:
review = df["Review"][2]
overall_chain.invoke(review)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate this 6    Está lu bonita calienta muy rápido, es muy fun...
Name: Review, dtype: object into spanish.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Based on Esta parrilla es bonita y se calienta muy rápido, es muy divertida..., summarize this review in less than 12 words in spanish.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate Parrilla bonita, rápida y divertida, se calienta rápido. into english

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Rate the quality of the translation of Parrilla bonita, rápida y divertida, se calienta rápido. into Nice, fast, and fun grill, it heats up quickly. from zero to ten.

> Finished chain.

> Finished chain.


{'review': 6    Está lu bonita calienta muy rápido, es muy fun...
 Name: Review, dtype: object,
 'review_es_short': 'Parrilla bonita, rápida y divertida, se calienta rápido.',
 'review_en_short': 'Nice, fast, and fun grill, it heats up quickly.',
 'translation_rating': 'I would rate the quality of the translation as an 8 out of 10. The overall meaning of the original phrase is captured well in the translation, but there could be some minor improvements in the wording to make it sound more natural in English.'}

**Repeat the above twice for different products or reviews**

In [56]:
review = df["Review"][4]
overall_chain.invoke(review)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate this  I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck. into spanish.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Based on Me encantó este producto. Pero parece que solo duran unos cuantos meses. La compañía fue genial reemplazando la primera (el espumador se cae del mango y no se puede arreglar). Después de 4 meses, mi segunda unidad hizo lo mismo. Solo uso el espumador para el café una vez al día. No es un uso excesivo o abusivo. Estoy muy decepcionado y buscaré otro

{'review': "\xa0I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
 'review_es_short': 'Producto bueno pero durabilidad corta, servicio al cliente útil pero limitado.',
 'review_en_short': 'Good product but short durability, helpful but limited customer service.',
 'translation_rating': 'I would rate the translation a 9 out of 10. It accurately conveys the original meaning in a clear and concise manner.'}

In [57]:
review = df["Review"][6]
overall_chain.invoke(review)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate this Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento. into spanish.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Based on Esta estufa es muy bonita y se calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento., summarize this review in less than 12 words in spanish.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate Estufa bonita, calienta rápido, funcional, durabilidad por verificar (3 días). into english

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Rate the quality of the translation of Estufa bonita, calienta rápido, funcional, durabilidad por verificar (3 días). into Beautiful stove, heats up quickly, functional, durability t

{'review': 'Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento.',
 'review_es_short': 'Estufa bonita, calienta rápido, funcional, durabilidad por verificar (3 días).',
 'review_en_short': 'Beautiful stove, heats up quickly, functional, durability to be verified (3 days).',
 'translation_rating': 'I would rate the quality of the translation a 7 out of 10. The overall meaning of the sentence is accurately conveyed, but there are some minor issues with the word choice and phrasing.'}

## Router Chain

In [58]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [76]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "history", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

prompt_templates = {
    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
    for info in prompt_infos
}


In [77]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [78]:
llm = ChatOpenAI(temperature=0)

In [79]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [80]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [81]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [82]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

#router_chain = router_prompt | llm | RunnableLambda(lambda x: x.content) 
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, 
                         verbose=True
                        )

In [84]:
chain.invoke("Explain what principle component analysis is using linear algebra")



> Entering new MultiPromptChain chain...
math: {'input': 'Explain what principle component analysis is using linear algebra'}
> Finished chain.


{'input': 'Explain what principle component analysis is using linear algebra',
 'text': "Principal component analysis (PCA) is a technique used in statistics and data analysis to reduce the dimensionality of a dataset while preserving as much variance as possible. In linear algebra terms, PCA involves finding the eigenvectors and eigenvalues of the covariance matrix of the dataset.\n\nTo perform PCA using linear algebra, follow these steps:\n\n1. Standardize the data: Subtract the mean from each feature and divide by the standard deviation to ensure that all features have the same scale.\n\n2. Compute the covariance matrix: Calculate the covariance matrix of the standardized data. The covariance matrix is a square matrix where each element represents the covariance between two features.\n\n3. Find the eigenvectors and eigenvalues: Compute the eigenvectors and eigenvalues of the covariance matrix. The eigenvectors represent the directions of maximum variance in the data, while the eigen

In [85]:
chain.invoke("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


{'input': 'What is black body radiation?',
 'text': "Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."}

In [86]:
chain.invoke("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


{'input': 'what is 2 + 2', 'text': 'The answer to 2 + 2 is 4.'}

In [87]:
chain.invoke("What is the name of the last dictator of Spain?")



> Entering new MultiPromptChain chain...
history: {'input': 'What is the name of the last dictator of Spain?'}
> Finished chain.


{'input': 'What is the name of the last dictator of Spain?',
 'text': 'The last dictator of Spain was Francisco Franco, who ruled as the head of state from 1939 until his death in 1975.'}

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [92]:
biology_template = """
You are an expert biologist with a deep understanding of cellular biology,\
genetics, ecology, and evolutionary theory. You explain complex biological concepts\
in a simple and accessible way for students or curious learners.\
When you don't know the answer, you are honest about it.
Here is a question:
{input}
"""

philosophy_template = """
You are a philosopher who specializes in metaphysics.\
You excel at explaining deep, abstract concepts such as the nature of existence,\
reality, and time in a way that a curious individual can understand.

Here is a question for you:
{input}
"""
prompt_infos.append({
    "name": "biology", 
    "description": "Good for answering biology questions", 
    "prompt_template": biology_template
    })
prompt_infos.append({
    "name": "philosophy", 
    "description": "Good for answering philosophical questions", 
    "prompt_template": philosophy_template
})

prompt_templates = {
    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
    for info in prompt_infos
}

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, 
                         verbose=True
                        )

In [93]:
chain.invoke("What is the concept of time in metaphysics?")



> Entering new MultiPromptChain chain...
philosophy: {'input': 'What is the concept of time in metaphysics?'}
> Finished chain.


{'input': 'What is the concept of time in metaphysics?',
 'text': 'In metaphysics, the concept of time is a complex and intriguing topic that has been debated by philosophers for centuries. Time is often seen as a fundamental aspect of reality, influencing our perception of existence and the world around us.\n\nOne common view of time in metaphysics is that it is a linear progression of events, with the past, present, and future all existing simultaneously in some sense. This view suggests that time is not just a series of moments passing by, but rather a fixed structure that we move through.\n\nAnother perspective on time in metaphysics is that it is a subjective experience, shaped by our consciousness and perception. This view suggests that time is not an objective reality, but rather a construct of our minds that helps us make sense of the world.\n\nOverall, the concept of time in metaphysics is a complex and multifaceted topic that raises important questions about the nature of rea

In [95]:
chain.invoke("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


{'input': 'Why does every cell in our body contain DNA?',
 'text': 'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for how our cells function and develop. DNA contains the information needed to make proteins, which are essential for the structure and function of our cells. Additionally, DNA is passed down from generation to generation, so it is present in every cell to ensure that the genetic information is maintained and passed on to future generations. This is why every cell in our body contains DNA.'}